In [1]:
import numpy as np
import scipy.linalg as sla
import numpy.linalg as la
from scipy.special import expit as sigmoid
import scipy as sc
from statsmodels.tsa.api import VAR

In [2]:
varables = 4
observations = 1000
window = 2
varTime = varables*window
#to use later 
times = 10

In [3]:
def qDag(w):
    return np.trace(sc.linalg.expm(np.multiply(w,w))) - w.shape[0]
def get_r(off):
    return lambda n: np.random.rand(n)+off

In [4]:
dagY = sc.sparse.random(varables, varables*window, density=0.15, data_rvs=get_r(2)).A
np.fill_diagonal(dagY, 0)

dagX = np.eye(varables, varables)
while qDag(dagX) != 0:
    dagX = sc.sparse.random(varables, varables, density=0.45, data_rvs=get_r(2)).A
    np.fill_diagonal(dagX, 0)

In [5]:
print(qDag(dagX))
dagX

0.0


array([[0.        , 2.31452364, 0.        , 2.16869477],
       [0.        , 0.        , 0.        , 2.26662303],
       [2.55705722, 2.36324595, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [6]:
def rand(n):
    return 2*(np.random.rand(n)-.5)

Y_pre = rand(observations*varables*window).reshape(varables*window, observations)
print(dagY.shape)
print(Y_pre.shape)

(4, 8)
(8, 1000)


In [7]:
#X = XW + YA + Z

def randFlat(X):
    return rand(X.shape[0]*X.shape[1]).reshape(*X.shape)

def update(preY, preX, dagY, dagX, rand):
    Xy = np.matmul(dagY, preY)
    Xr = rand(preX)
    Xx = np.matmul(dagX, Xr)
    X = Xr + Xx + Xy
    
    return X

def step(preY, dagY, dagX, window, var, rand):
    preX = preY[:var]
    preY = preY[:window*var]
    return update(preY, preX, dagY, dagX, rand)

def makeSynthData(preY, times, obs, var, pad, rand=rand):
    window = preY.shape[0]//var
    
    #for i in range(pad+window):
     #   newX = step(preY, dagY, dagX, window, var, rand)
      #  preY = np.append(newX, preY, axis=0)
        
    Y = Y_pre[:window*var]

    for i in range(times - window):
        newX = step(Y, dagY, dagX, window, var, rand)
        Y = np.append(newX, Y, axis=0)
        
    return Y
    



#later
#W = np.random.rand(varables*observations).reshape(varables,observations)

In [8]:
XY = makeSynthData(Y_pre, 6, observations, varables, 0, randFlat)


In [9]:
Y=XY[varables:window*varables+varables]
X=XY[:varables]

In [10]:
X.shape

(4, 1000)

In [11]:
C_init = np.random.rand(varables*varables*window + varables*varables)

In [12]:
def f(X, Y, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(varables, varables*window)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/X.shape[1]
        #print()
        #print(loss)
        #print(np.square(np.linalg.norm(X)))
        #print(np.square(np.linalg.norm(X - np.matmul(w, X))))
        #print(np.square(np.linalg.norm(X - np.matmul(a, Y))))
        #print(np.linalg.norm(w.flatten(), 1))
        #print(np.linalg.norm(a.flatten(), 1))
    
        return loss + lB*np.linalg.norm(w.flatten(), 1)# + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

In [13]:
def constr(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr}

C = sc.optimize.minimize(f(X,Y,1,1) , C_init, constraints=cons).x

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/linalg/_matfuncs.py:378: RuntimeWarning: overflow encountered in matmul
  eAw = eAw @ eAw


In [14]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [15]:
A[A<.3] = 0
A.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 2.2, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 3. , 2.1, 2.4, 2.3]])

In [16]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 2.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 2.9, 2.2, 2.4, 2.3]])

In [17]:
W.round(1)

array([[ 0. ,  0.9,  0. ,  0. ],
       [ 0. ,  0. , -0. ,  0.8],
       [ 0.7, -0.1,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ]])

In [18]:
dagX.round(1)

array([[0. , 2.3, 0. , 2.2],
       [0. , 0. , 0. , 2.3],
       [2.6, 2.4, 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

In [19]:
dagX.T.round(1)

array([[0. , 0. , 2.6, 0. ],
       [2.3, 0. , 2.4, 0. ],
       [0. , 0. , 0. , 0. ],
       [2.2, 2.3, 0. , 0. ]])

In [43]:
def metrics(A,dagY, W, dagX):
    A = np.absolute(A) > 0.3
    dagY = np.absolute(dagY) > .5
    W = np.absolute(W) > 0.3
    dagX = np.absolute(dagX) > .5
    
    preWrong = A != dagY
    wWrong = W != dagX
    #assumes cant be both wrong and inv
    wInverse = W.T & dagX
    
    if ((W + W.T)==2).any():
        print(W)
        print(W.T)
        raise "error"
    
    #print(A)
    #print(dagY)
    #print(W)
    #print(dagX)
    #print()
    #print(preWrong)
    #print(wWrong)
    #print(wInverse)
    
    return {
        "preWrong": sum(sum(preWrong)),
        "wWrong": sum(sum(wWrong)),
        "wInverse": sum(sum(wInverse))
    }
    

In [44]:
metrics(A,dagY, W, dagX)

[[False False False False False False False False]
 [False False False False False False False False]
 [False False False False  True False False False]
 [False False False False  True  True  True  True]]
[[False False False False False False False False]
 [False False False False False False False False]
 [False False False False  True False False False]
 [False False False False  True  True  True  True]]
[[False  True False False]
 [False False False  True]
 [ True False False False]
 [False False False False]]
[[False  True False  True]
 [False False False  True]
 [ True  True False False]
 [False False False False]]

[[False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]]
[[False False False  True]
 [False False False False]
 [False  True False False]
 [False False False False]]
[[False False False False]
 [False False False False]
 [F

{'preWrong': 0, 'wWrong': 2, 'wInverse': 0}

In [35]:
A.round()

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 3., 2., 2., 2.]])

In [36]:
dagY.round()

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 3., 2., 2., 2.]])

In [37]:
np.matmul(np.eye(varables, varables) - W, X).round(1)

array([[ -1.6,  -0.7,   0.2, ...,  -0.5,  -0. ,   0.8],
       [-11.3, -13.6,  -8.6, ...,  -1.8,  16. ,   5.8],
       [  1.5,   2. ,   0.5, ...,   3.2,  -7.2,  -0.3],
       [ 14.7,  18.7,  10.6, ...,   4.2, -21.1,  -9.1]])

In [38]:
X

array([[-1.53734929e+00,  7.29772421e-03, -1.89720460e-01, ...,
         7.92127115e-01, -2.70753545e-01, -2.32432737e-01],
       [ 1.52326956e-02,  7.91472774e-01, -4.53883454e-01, ...,
         1.37400446e+00, -2.73992203e-01, -1.11320777e+00],
       [ 4.50278738e-01,  1.95999831e+00,  3.74543173e-01, ...,
         3.60226271e+00, -7.36021715e+00, -3.79757880e-01],
       [ 1.46815545e+01,  1.86895442e+01,  1.05568866e+01, ...,
         4.17700824e+00, -2.11307550e+01, -9.05476445e+00]])